In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# if using mac
df = pd.read_excel('/Users/zhanghantang/PycharmProjects/sucide_project/BIOM40forUSC.xlsx')
data = df[df['SI'].notnull()]
y = data['SI']
x = data.loc[:, 'GIMAP1Biom1552316_a_at':'CFIS']

In [3]:
# column 'CFI-S.PheneVisit' data type is string, i am not clear its internal meaning and how to convert to float data type
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split
x = x.drop(labels='CFI-S.PheneVisit', axis=1)
# drop these column directly
x = data.loc[:, 'GIMAP1Biom1552316_a_at':'RAB3GAP2Biom240234_at']
y = np.array(y)
X = StandardScaler().fit_transform(x)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)
print(len(X_train), len(X_test))

# todo unified randome state
unified_random_state = 32
y_list = list(y)
print(y_list.count(0), y_list.count(1), y_list.count(2), y_list.count(3), y_list.count(4))

603 151
547 64 49 57


In [5]:
'''
-------------------------
the content below is about logistic regression content
-------------------------
'''

547

In [63]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='saga', max_iter=5000, )
# 不同的solver可能只支持不同的有限的penalty
param_grid = {'penalty':['l1', 'l2'], 'C':[0.1, 1, 10, 100], 'multi_class':['ovr', 'multinomial']}

# The macro-averaged F1 score gives equal weight to each class, best metric for our data
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='f1_macro', return_train_score=True)
grid_search.fit(X, y)
print("Best penalty: ", grid_search.best_params_['penalty'])
print("Best C: ", grid_search.best_params_['C'])
print("Best score: ", grid_search.best_score_)

Best penalty:  l1
Best C:  10
Best score:  0.2006970876816408


In [76]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LogisticRegression as LR
from sklearn.metrics import make_scorer, roc_auc_score
# scoring = ['accuracy', 'precision', 'recall', 'f1_macro', 'roc_auc']
# scoring = {
#     'accuracy': 'accuracy',
#     'precision_macro':make_scorer(precision_score, zero_division=0, average='macro'),
#     'recall_macro' :make_scorer(recall_score, average='macro'),
#     'f1_macro':'f1_macro',
#     'roc_auc':'roc_auc'
# }
scoring = {
    'accuracy': 'accuracy',
    'f1_macro':'f1_macro',
}
lr_model = LR(solver='saga', penalty='l1', C=10, max_iter=5000, multi_class='multinomial')
scores = cross_validate(lr_model, X, y, cv=5, scoring=scoring)
# print("Scores: accuracy is {}, mean accuary is {}".format(scores['test_accuracy'], scores['test_accuracy'].mean()))
# print("Scores: precision is {}, mean precision is {}".format(scores['test_precision'], scores['test_precision'].mean()))
# print("Scores: recall is {}, mean recall is {}".format(scores['test_recall'], scores['test_recall'].mean()))
# print("Scores: f1 is {}, mean f1 is {}".format(scores['test_f1'], scores['test_f1_macro'].mean()))
# print("Scores: roc_auc is {}, mean roc_auc is {}".format(scores['test_roc_auc'], scores['test_roc_auc'].mean()))
print("Scores: accuracy is {}, mean accuary is {}".format(scores['test_accuracy'], scores['test_accuracy'].mean()))
print("Scores: f1 is {}, mean f1 is {}".format(scores['test_f1_macro'], scores['test_f1_macro'].mean()))


'''
Scores: accuracy is [0.64900662 0.68874172 0.63576159 0.69536424 0.64      ], mean accuary is 0.6617748344370861
Scores: f1 is [0.18568306 0.16377953 0.18121804 0.220515   0.25228982], mean f1 is 0.2006970876816408


'''

Scores: accuracy is [0.64900662 0.68874172 0.63576159 0.69536424 0.64      ], mean accuary is 0.6617748344370861
Scores: f1 is [0.18568306 0.16377953 0.18121804 0.220515   0.25228982], mean f1 is 0.2006970876816408


'\nScores: accuracy is [0.64900662 0.68874172 0.63576159 0.69536424 0.64      ], mean accuary is 0.6617748344370861\nScores: f1 is [0.18568306 0.16377953 0.18121804 0.220515   0.25228982], mean f1 is 0.2006970876816408\n\n\n'

In [ ]:
'''
-------------------------
the content below is about SVM
-------------------------
'''

In [79]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate

model = SVC(kernel='rbf', max_iter=10000, class_weight='balanced')
param_grid = {'C':[100, 150, 200], }
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='f1_macro', return_train_score=True)
grid_search.fit(X, y)

# print("Best penalty: ", grid_search.best_params_['kernel'])
print("Best C: ", grid_search.best_params_['C'])
print("Best score: ", grid_search.best_score_)

'''
Best penalty:  rbf
Best C:  100

Scores:  [0.36065574 0.44067797 0.45901639 0.31111111 0.40740741]
Mean Score:  0.39577372315355086
Scores:  [0.74274194 0.71833931 0.72649573 0.70439633 0.69226137]
Mean Score:  0.7168469323092028

'''

Best C:  150
Best score:  0.24237674223354908


'\nBest penalty:  rbf\nBest C:  100\n\nScores:  [0.36065574 0.44067797 0.45901639 0.31111111 0.40740741]\nMean Score:  0.39577372315355086\nScores:  [0.74274194 0.71833931 0.72649573 0.70439633 0.69226137]\nMean Score:  0.7168469323092028\n\n'

In [93]:
model = SVC(kernel='rbf', max_iter=5000, class_weight='balanced', C=150)
scoring = {
    'accuracy': 'accuracy',
    'f1_macro':'f1_macro',
}
scores = cross_validate(model, X, y, cv=5, scoring=scoring)
print("Scores: accuracy is {}, mean accuary is {}".format(scores['test_accuracy'], scores['test_accuracy'].mean()))
print("Scores: f1 is {}, mean f1 is {}".format(scores['test_f1_macro'], scores['test_f1_macro'].mean()))

'''
Scores: accuracy is [0.64900662 0.66225166 0.63576159 0.64900662 0.50666667], mean accuary is 0.6205386313465784
Scores: f1 is [0.24741249 0.22642792 0.22149303 0.24772155 0.26696575], mean f1 is 0.242004147773067

'''

Scores: accuracy is [0.64900662 0.66225166 0.63576159 0.64900662 0.51333333], mean accuary is 0.6218719646799117
Scores: f1 is [0.24741249 0.22642792 0.22244541 0.24700767 0.2695426 ], mean f1 is 0.2425672184240253


'\nScores: accuracy is [0.64900662 0.66225166 0.63576159 0.64900662 0.50666667], mean accuary is 0.6205386313465784\nScores: f1 is [0.24741249 0.22642792 0.22149303 0.24772155 0.26696575], mean f1 is 0.242004147773067\n\n'

In [ ]:
'''
-------------------------
the content below is about random forest
-------------------------
'''

In [5]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold

param_grid = {'n_estimators':[10, 100, 200], 'max_depth':[8, 11, 13], 'min_samples_split': [4, 6, 8, 10],
              'min_samples_leaf': [2, 4, 6]}

skf = StratifiedKFold(n_splits=5)
rf_model = RandomForestClassifier(random_state=unified_random_state)
grid_search = GridSearchCV(rf_model, param_grid, cv=skf, scoring='f1_macro')
grid_search.fit(X, y)

print(grid_search.best_params_)

{'max_depth': 11, 'min_samples_leaf': 2, 'min_samples_split': 8, 'n_estimators': 10}


In [12]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, make_scorer

scoring = {
    'accuracy': 'accuracy',
    'f1_macro':'f1_macro',
}

rf_model = RandomForestClassifier(n_estimators=10, max_depth=11, min_samples_leaf=2, min_samples_split=8, random_state=unified_random_state)

# scoring = ['accuracy', 'precision', 'recall', 'f1', 'roc_auc']

scores = cross_validate(rf_model, X, y, cv=5, scoring=scoring)
print("Scores: accuracy is {}, mean accuary is {}".format(scores['test_accuracy'], scores['test_accuracy'].mean()))
print("Scores: f1 is {}, mean f1 is {}".format(scores['test_f1_macro'], scores['test_f1_macro'].mean()))

# print("Score with L1 penalty: accuracy:{} precision:{} recall:{} f1:{} roc_auc:{}".format(accuracy_score(y_test, y_pred), precision_score(y_test, y_pred), recall_score(y_test, y_pred), f1_score(y_test, y_pred), roc_auc_score(y_test, y_pred)))
'''
Scores: accuracy is [0.7218543  0.7218543  0.73509934 0.72847682 0.70666667], mean accuary is 0.7227902869757175
Scores: f1 is [0.16769231 0.16833977 0.19911406 0.16988417 0.25703768], mean f1 is 0.19241359726083926

'''

Scores: accuracy is [0.71523179 0.7218543  0.73509934 0.72847682 0.7       ], mean accuary is 0.7201324503311259
Scores: f1 is [0.19153696 0.16833977 0.2006534  0.17054264 0.23872815], mean f1 is 0.19396018441926785


'\nScores: accuracy is [0.7218543  0.7218543  0.73509934 0.72847682 0.70666667], mean accuary is 0.7227902869757175\nScores: f1 is [0.16769231 0.16833977 0.19911406 0.16988417 0.25703768], mean f1 is 0.19241359726083926\n\n'

In [16]:
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold
clf = XGBClassifier(random_state=unified_random_state, objective='multi:softmax', num_class=5, )
param_grid = {'n_estimators':[10, 100, 200], 'max_depth':[4, 6, 8], 'learning_rate': [0.3, 0.01]}
grid_search = GridSearchCV(clf, param_grid, cv=5, scoring='f1_macro')
grid_search.fit(X, y)

print(grid_search.best_params_)

{'learning_rate': 0.01, 'max_depth': 8, 'n_estimators': 100}


In [43]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, make_scorer

scoring = {
    'accuracy': 'accuracy',
    'f1_macro':'f1_macro',
}

clf = XGBClassifier(n_estimators=100, learning_rate=0.01, max_depth=8, subsample=1)
scores = cross_validate(clf, X, y, cv=5, scoring=scoring)
print("Scores: accuracy is {}, mean accuary is {}".format(scores['test_accuracy'], scores['test_accuracy'].mean()))
print("Scores: f1 is {}, mean f1 is {}".format(scores['test_f1_macro'], scores['test_f1_macro'].mean()))

'''
Scores: accuracy is [0.69536424 0.70198675 0.7218543  0.71523179 0.62      ], mean accuary is 0.6908874172185431
Scores: f1 is [0.24137887 0.22710069 0.24285714 0.22783049 0.2089161 ], mean f1 is 0.22961665754520685
'''

Scores: accuracy is [0.62913907 0.68874172 0.73509934 0.69536424 0.61333333], mean accuary is 0.672335540838852
Scores: f1 is [0.17405797 0.22587977 0.27777778 0.22605263 0.22640889], mean f1 is 0.22603540619990542


'\nScores: accuracy is [0.69536424 0.70198675 0.7218543  0.71523179 0.62      ], mean accuary is 0.6908874172185431\nScores: f1 is [0.24137887 0.22710069 0.24285714 0.22783049 0.2089161 ], mean f1 is 0.22961665754520685\n'

In [ ]:
'''
-------------------------
the content below is about native bayes
-------------------------
'''

In [46]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_validate
from sklearn.naive_bayes import GaussianNB

# skf = StratifiedKFold(n_splits=5)

nb = GaussianNB()
scoring = {
    'accuracy': 'accuracy',
    'f1_macro':'f1_macro',
}

scores = cross_validate(nb, X, y, cv=5, scoring=scoring)
print("Scores: accuracy is {}, mean accuary is {}".format(scores['test_accuracy'], scores['test_accuracy'].mean()))
print("Scores: f1 is {}, mean f1 is {}".format(scores['test_f1_macro'], scores['test_f1_macro'].mean()))

'''
Scores: accuracy is [0.56291391 0.54966887 0.37086093 0.2384106  0.38      ], mean accuary is 0.42037086092715226
Scores: precision is [0.1509434  0.24675325 0.17       0.16793893 0.20183486], mean precision is 0.18749408733253856
Scores: recall is [0.27586207 0.65517241 0.5862069  0.78571429 0.78571429], mean recall is 0.6177339901477832
Scores: f1 is [0.19512195 0.35849057 0.26356589 0.27672956 0.32116788], mean f1 is 0.2830151703380446
Scores: roc_auc is [0.4535048  0.58988129 0.45293951 0.44976771 0.53629977], mean roc_auc is 0.4964786170917511

'''

Scores: accuracy is [0.2781457  0.21854305 0.15231788 0.11258278 0.16      ], mean accuary is 0.184317880794702
Scores: f1 is [0.19394946 0.12108522 0.13812573 0.08840049 0.1179354 ], mean f1 is 0.1318992599341135


'\nScores: accuracy is [0.56291391 0.54966887 0.37086093 0.2384106  0.38      ], mean accuary is 0.42037086092715226\nScores: precision is [0.1509434  0.24675325 0.17       0.16793893 0.20183486], mean precision is 0.18749408733253856\nScores: recall is [0.27586207 0.65517241 0.5862069  0.78571429 0.78571429], mean recall is 0.6177339901477832\nScores: f1 is [0.19512195 0.35849057 0.26356589 0.27672956 0.32116788], mean f1 is 0.2830151703380446\nScores: roc_auc is [0.4535048  0.58988129 0.45293951 0.44976771 0.53629977], mean roc_auc is 0.4964786170917511\n\n'

In [ ]:
'''
-------------------------
the content below is about dnn
-------------------------
'''

In [64]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import f1_score, accuracy_score
from torch.utils.data import DataLoader, TensorDataset
from torch.optim.lr_scheduler import ReduceLROnPlateau


# Define evaluation function
def evaluate(model, criterion, dataloader, device):
    model.eval()
    running_loss = 0.0
    y_true, y_pred = [], []
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)
            predicted = torch.argmax(outputs, dim=1)
            y_true.extend(labels.tolist())
            y_pred.extend(predicted.tolist())

    epoch_loss = running_loss / len(dataloader.dataset)
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, zero_division=1.0, average='macro')
    recall = recall_score(y_true, y_pred, average='macro')
    f1 = f1_score(y_true, y_pred, average='macro')
    roc_auc = roc_auc_score(y_true, y_pred, average='macro', multi_class='ovr')

    return epoch_loss, accuracy, precision, recall, f1, roc_auc

# Training loop
def train(model, criterion, optimizer, scheduler, train_dataloader, val_dataloader, device, num_epochs=10):
    # best_val_loss = float('inf')
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)

        epoch_loss = running_loss / len(train_dataloader.dataset)
        val_loss, accuracy, precision, recall, f1, roc_auc = evaluate(model, criterion, val_dataloader, device)
        scheduler.step(val_loss)

        print(f'Epoch {epoch+1}/{num_epochs}, '
              f'Training Loss: {epoch_loss:.4f}, '
              f'Validation Loss: {val_loss:.4f}, '
              f'Accuracy: {accuracy:.4f}, '
              f'Precision: {precision:.4f}, '
              f'Recall: {recall:.4f}, '
              f'F1-score: {f1:.4f}, '
              f'ROC AUC: {roc_auc:.4f}')

        # if val_loss < best_val_loss:
        #     best_val_loss = val_loss
        #     torch.save(model.state_dict(), 'best_model.pth')

class MyDataset(torch.utils.data.Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [48]:
# Set the minimum number of positive samples per split will reduce the performance

X_dnn = torch.tensor(X).float()
y_dnn = torch.tensor(y).float()
# Split the data into training and validation sets
dataset = TensorDataset(X_dnn, y_dnn)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=True)

In [62]:
test = X_dnn[0:5]
input_size = x.shape[1]
net = BinaryClassifier(input_size)
outputs = net(test)
labels = y_dnn[0:5].long()
print(outputs)
print(y_dnn[0:5])
criterion = nn.CrossEntropyLoss()
loss = criterion(outputs, labels)
print(loss)
preds = torch.argmax(outputs, dim=1)
print(preds)

tensor([[-0.0538,  0.1139, -0.0963,  0.0773,  0.0005],
        [-0.0863,  0.1259, -0.0721,  0.0988, -0.0041],
        [-0.0732,  0.1252, -0.0851,  0.0873, -0.0095],
        [-0.0584,  0.1138, -0.0897,  0.0835, -0.0038],
        [-0.0603,  0.1211, -0.0931,  0.0803, -0.0017]],
       grad_fn=<AddmmBackward0>)
tensor([3., 0., 0., 3., 1.])
tensor(1.5979, grad_fn=<NllLossBackward0>)
tensor([1, 1, 1, 1, 1])


In [ ]:
input_size = x.shape[1]

class BinaryClassifier(nn.Module):
    def __init__(self, input_size):
        super(BinaryClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 64)
        self.fc5 = nn.Linear(64, 32)
        # self.sigmoid = nn.Sigmoid()
        self.out = nn.Linear(32, 5)
        self.relu = nn.ReLU()
        self.dropout1 = nn.Dropout(p=0.2)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout1(x)
        x = self.relu(self.fc2(x))
        x = self.dropout1(x)
        x = self.relu(self.fc3(x))
        x = self.relu(self.fc4(x))
        x = self.relu(self.fc5(x))
        out = self.out(x)
        return out

model = BinaryClassifier(input_size)
# commonly use in binary classification
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print(device)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=400, verbose=True)
train(model, criterion, optimizer, scheduler, train_loader, val_loader, device, num_epochs=500)


'''
self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 32)
        self.fc5 = nn.Linear(32, 1)
drop 0.2 2 layers
Training Loss: 0.0025, Validation Loss: 1.9067, Accuracy: 0.8013, Precision: 0.3000, Recall: 0.5000, F1-score: 0.3750, ROC AUC: 0.6711


self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 64)
        self.fc5 = nn.Linear(64, 32)
        self.fc6 = nn.Linear(32, 1)
Training Loss: 0.0005, Validation Loss: 1.3775, Accuracy: 0.8146, Precision: 0.3333, Recall: 0.5556, F1-score: 0.4167, ROC AUC: 0.7026
'''